In [5]:
import time
import cv2 #to work with images from cameras
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
client = carla.Client('localhost', 2000)
world = client.get_world()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

CAMERA_POS_Z = 2
CAMERA_POS_X = 2 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

# define speed contstants
PREFERRED_SPEED = 30 # what it says
PREFERRED_SPEED2 = 15 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 50


def collision_check(point, obstacles):
    collision_free = True
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<15:
            collision_free=False
            break

    return collision_free
def get_short_dist(point,obstacles):
    min_dist=float('inf')
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<min_dist:
            min_dist=collision_dist

    return min_dist

def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    c=math.cos(car.get_transform().rotation.yaw*math.pi/180)
    s=math.sin(car.get_transform().rotation.yaw*math.pi/180)
    for detection in point_cloud_data:
        #l.append([round(detection.point.x+car.get_location().x,1),round(detection.point.y+car.get_location().y,1)])
        #l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])
        l.append([round((detection.point.x *c +detection.point.y*-s)+car.get_location().x,1),round((detection.point.x*s+detection.point.y*c)+car.get_location().y,1)])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(1000))
    lidar_blueprint.set_attribute("range",str(50))
    lidar_blueprint.set_attribute("upper_fov",str(1))
    lidar_blueprint.set_attribute("lower_fov",str(-1))
#    lidar_blueprint.set_attribute("horizontal_fov",str(360))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 
def maintain_speed2(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED2:
        return 0
    elif s < PREFERRED_SPEED2 - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

wa=world.get_map().get_waypoint(carla.Location(x=-60,y=137,z=0.5))
start_point=wa.transform
start_point.location.z=0.5
midle_point=wa.transform
midle_point.location.z=0.5
midle_point.location.x=-30

vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle2 = world.try_spawn_actor(vehicle_bp[0], midle_point)

get_blueprint_of_world = world.get_blueprint_library()
lidar_sensor = generate_lidar_blueprint(get_blueprint_of_world)
sensor_lidar_spawn_point = carla.Transform(carla.Location(x=0, y=0, z=1.2),
                                           carla.Rotation(pitch=0.000000, yaw=0.0, roll=0.000000))
sensor = world.spawn_actor(lidar_sensor, sensor_lidar_spawn_point, attach_to=vehicle)
wa=world.get_map().get_waypoint(carla.Location(x=60,y=137,z=0.5))
end_point=wa.transform.location

route = grp.trace_route(start_point.location, end_point)

curIndx=1

curr_wp = 1

ladarData=[]
sensor.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data,vehicle,ladarData))

ladarData1=ladarData.copy()

camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('depth Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('depth Camera',camera_data['image'])
flag=0
BRAKE=0
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
while curr_wp<len(route)-11:
    
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    new_image=depth_to_array(image)
    ladarData1=ladarData.copy()

    xf=len(ladarData1)
    for i in range(xf):
        if -ladarData1[xf-1-i][0]+vehicle.get_location().x>3 or abs(ladarData1[xf-1-i][1]-vehicle.get_location().y)>3:
            ladarData1.pop(xf-1-i)

#    for POINT in ladarData1:
#        world.debug.draw_string(carla.Location(x=POINT[0],y=POINT[1]), '^', draw_shadow=False,
#            color=carla.Color(r=0, g=0, b=255), life_time=1.0,
#            persistent_lines=True) 

    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close

    v = vehicle.get_velocity()
    v2 = vehicle2.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    speed2 = round(3.6 * math.sqrt(v2.x**2 + v2.y**2 + v2.z**2),0)
    #image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    estimated_throttle2 = maintain_speed2(speed2)
    steer_input = 0
    steer_input = steer_input/75
    vehicle2.apply_control(carla.VehicleControl(throttle=estimated_throttle2, steer=0))
    if get_short_dist([vehicle.get_location().x,vehicle.get_location().y],ladarData1)<20:
        if flag==1:
            start_dist=end_dist
            strat_time=end_time
            end_dist=get_short_dist([vehicle.get_location().x,vehicle.get_location().y],ladarData1)
            end_time=time.time()

            PREFERRED_SPEED=((end_dist-start_dist)/(end_time-strat_time))+speed
            if PREFERRED_SPEED+5<speed:
                estimated_throttle=0
                BRAKE=1
            else:
                BRAKE=0
        end_dist=get_short_dist([vehicle.get_location().x,vehicle.get_location().y],ladarData1)
        end_time=time.time()
        flag=1
    else:
        PREFERRED_SPEED=50
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input,brake=BRAKE))
    cv2.imshow('depth Camera',new_image)


cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [36]:
cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [7]:
# Copyright (c) 2017 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

"""
Handy conversions for CARLA images.

The functions here are provided for real-time display, if you want to save the
converted images, save the images from Python without conversion and convert
them afterwards with the C++ implementation at "Util/ImageConverter" as it
provides considerably better performance.
"""

import math

try:
    import numpy
    from numpy.matlib import repmat
except ImportError:
    raise RuntimeError('cannot import numpy, make sure numpy package is installed')




def to_bgra_array(image):
    """Convert a CARLA raw image to a BGRA numpy array."""
    if not isinstance(image, sensor.Image):
        raise ValueError("Argument must be a carla.sensor.Image")
    array = numpy.frombuffer(image.raw_data, dtype=numpy.dtype("uint8"))
    array = numpy.reshape(array, (image.height, image.width, 4))
    return array


def to_rgb_array(image):
    """Convert a CARLA raw image to a RGB numpy array."""
    array = to_bgra_array(image)
    # Convert BGRA to RGB.
    array = array[:, :, :3]
    array = array[:, :, ::-1]
    return array


def labels_to_array(image):
    """
    Convert an image containing CARLA semantic segmentation labels to a 2D array
    containing the label of each pixel.
    """
    return to_bgra_array(image)[:, :, 2]


def labels_to_cityscapes_palette(image):
    """
    Convert an image containing CARLA semantic segmentation labels to
    Cityscapes palette.
    """
    classes = {
        0: [0, 0, 0],         # None
        1: [70, 70, 70],      # Buildings
        2: [190, 153, 153],   # Fences
        3: [72, 0, 90],       # Other
        4: [220, 20, 60],     # Pedestrians
        5: [153, 153, 153],   # Poles
        6: [157, 234, 50],    # RoadLines
        7: [128, 64, 128],    # Roads
        8: [244, 35, 232],    # Sidewalks
        9: [107, 142, 35],    # Vegetation
        10: [0, 0, 255],      # Vehicles
        11: [102, 102, 156],  # Walls
        12: [220, 220, 0]     # TrafficSigns
    }
    array = labels_to_array(image)
    result = numpy.zeros((array.shape[0], array.shape[1], 3))
    for key, value in classes.items():
        result[numpy.where(array == key)] = value
    return result


def depth_to_array(image):
    """
    Convert an image containing CARLA encoded depth-map to a 2D array containing
    the depth value of each pixel normalized between [0.0, 1.0].
    """
    array = image
    array = array.astype(numpy.float32)
    # Apply (R + G * 256 + B * 256 * 256) / (256 * 256 * 256 - 1).
    normalized_depth = numpy.dot(array[:, :, :3], [65536.0, 256.0, 1.0])
    normalized_depth /= 1000000.0  # (256.0 * 256.0 * 256.0 - 1.0)
    return normalized_depth


def depth_to_logarithmic_grayscale(image):
    """
    Convert an image containing CARLA encoded depth-map to a logarithmic
    grayscale image array.
    "max_depth" is used to omit the points that are far enough.
    """
    normalized_depth = depth_to_array(image)
    # Convert to logarithmic depth.
    logdepth = numpy.ones(normalized_depth.shape) + \
        (numpy.log(normalized_depth) / 5.70378)
    logdepth = numpy.clip(logdepth, 0.0, 1.0)
    logdepth *= 255.0
    # Expand to three colors.
    return numpy.repeat(logdepth[:, :, numpy.newaxis], 3, axis=2)


def depth_to_local_point_cloud(image,vehicle, color=None, max_depth=0.9):
    """
    Convert an image containing CARLA encoded depth-map to a 2D array containing
    the 3D position (relative to the camera) of each pixel and its corresponding
    RGB color of an array.
    "max_depth" is used to omit the points that are far enough.
    """
    far = 59.6  # max depth in meters.
    normalized_depth = depth_to_array(image)

    # (Intrinsic) K Matrix
    k = numpy.identity(3)
    k[0, 2] = 512 / 2.0
    k[1, 2] = 424 / 2.0
    k[0, 0] = k[1, 1] = 512 / \
        (2.0 * math.tan(70 * math.pi / 360.0))

    # 2d pixel coordinates
    pixel_length = 512 * 424
    u_coord = repmat(numpy.r_[512-1:-1:-1],
                     424, 1).reshape(pixel_length)
    v_coord = repmat(numpy.c_[424-1:-1:-1],
                     1, 512).reshape(pixel_length)
    if color is not None:
        color = color.reshape(pixel_length, 3)
    normalized_depth = numpy.reshape(normalized_depth, pixel_length)

    # Search for pixels where the depth is greater than max_depth to
    # delete them
    max_depth_indexes = numpy.where(normalized_depth > max_depth)
    normalized_depth = numpy.delete(normalized_depth, max_depth_indexes)
    u_coord = numpy.delete(u_coord, max_depth_indexes)
    v_coord = numpy.delete(v_coord, max_depth_indexes)
    if color is not None:
        color = numpy.delete(color, max_depth_indexes, axis=0)

    # pd2 = [u,v,1]
    p2d = numpy.array([u_coord, v_coord, numpy.ones_like(u_coord)])

    # P = [Y,Z,X]
    p3d = numpy.dot(numpy.linalg.inv(k), p2d)
    p3d *= normalized_depth * far
    p3d[0]*=-1
    p3d[2]+=vehicle.get_location().x+2
    p3d[0]+=vehicle.get_location().y
    p3d[1]+=vehicle.get_location().z+2
    # Formating the output to:
    # [[X1,Y1,Z1,R1,G1,B1],[X2,Y2,Z2,R2,G2,B2], ... [Xn,Yn,Zn,Rn,Gn,Bn]]
    if color is not None:
        # numpy.concatenate((numpy.transpose(p3d), color), axis=1)
        return sensor.PointCloud(
            image.frame_number,
            numpy.transpose(p3d),
            color_array=color)
    # [[X1,Y1,Z1],[X2,Y2,Z2], ... [Xn,Yn,Zn]]
    return numpy.transpose(p3d)

In [10]:
import time
import cv2 #to work with images from cameras
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
#import open3d as o3d

client = carla.Client('localhost', 2000)
world = client.get_world()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

CAMERA_POS_Z = 2
CAMERA_POS_X = 2 

# Depth camera parameters:
FX_DEPTH = 5.8262448167737955e+02
FY_DEPTH = 5.8269103270988637e+02
CX_DEPTH = 3.1304475870804731e+02
CY_DEPTH = 2.3844389626620386e+02

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

# define speed contstants
PREFERRED_SPEED = 30 # what it says
PREFERRED_SPEED2 = 15 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 50


def collision_check(point, obstacles):
    collision_free = True
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<15:
            collision_free=False
            break

    return collision_free
def get_short_dist(point,obstacles):
    min_dist=float('inf')
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<min_dist:
            min_dist=collision_dist

    return min_dist

def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    c=math.cos(car.get_transform().rotation.yaw*math.pi/180)
    s=math.sin(car.get_transform().rotation.yaw*math.pi/180)
    for detection in point_cloud_data:
        #l.append([round(detection.point.x+car.get_location().x,1),round(detection.point.y+car.get_location().y,1)])
        #l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])
        l.append([round((detection.point.x *c +detection.point.y*-s)+car.get_location().x,1),round((detection.point.x*s+detection.point.y*c)+car.get_location().y,1)])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(1000))
    lidar_blueprint.set_attribute("range",str(50))
    lidar_blueprint.set_attribute("upper_fov",str(1))
    lidar_blueprint.set_attribute("lower_fov",str(-1))
#    lidar_blueprint.set_attribute("horizontal_fov",str(360))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 
def maintain_speed2(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED2:
        return 0
    elif s < PREFERRED_SPEED2 - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

wa=world.get_map().get_waypoint(carla.Location(x=-60,y=137,z=0.5))
start_point=wa.transform
start_point.location.z=0.5
midle_point=wa.transform
midle_point.location.z=0.5
midle_point.location.x=-50

vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle2 = world.try_spawn_actor(vehicle_bp[0], midle_point)


camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
camera_bp.set_attribute('image_size_x', '512') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '424')
camera_bp.set_attribute('fov', '70')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('depth Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('depth Camera',camera_data['image'])
while True:
    
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    new_image=depth_to_array(image)
    POINTS=depth_to_local_point_cloud(image,vehicle,max_depth=0.2)
#    pcd = []
#    height, width = new_image.shape
#    for i in range(height):
#       for j in range(width):
#           z = new_image[i][j]
#           x = (j - CX_DEPTH) * z / FX_DEPTH
#           y = (i - CY_DEPTH) * z / FY_DEPTH
#           pcd.append([x, y, z])
    
#    for POINT in pcd:
#        world.debug.draw_string(carla.Location(x=POINT[0],y=POINT[1],z=POINT[2]), '^', draw_shadow=False,
#            color=carla.Color(r=0, g=0, b=255), life_time=0.00001,
#            persistent_lines=True) 

#    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
#    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    # Visualize:
#    o3d.visualization.draw_geometries([pcd_o3d])

    new_image2=new_image.copy()
    for x in range(len(new_image2)):
        for y in range(len(new_image2[x])):
            if new_image2[x][y]<0.2 and x<280:
                new_image2[x][y]=16

    min_val=float('inf')
    for x in range(len(new_image)):
        for y in range(len(new_image[x])):
            if new_image[x][y]<min_val and x<280:
                min_val=new_image[x][y]

    #image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('depth Camera',new_image2)

#print(POINTS)
for POINT in POINTS:
    if POINT[1]>0.5 and abs(POINT[0]-vehicle.get_location().y)<2:
        world.debug.draw_string(carla.Location(x=POINT[2],y=POINT[0],z=POINT[1]), '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=30,
            persistent_lines=True) 

#for POINT in POINTS:
#    print(POINT)

cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [1]:
import carla #the sim library itself
client = carla.Client('localhost', 2000)
world = client.get_world()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [8]:
import time
import cv2 #to work with images from cameras
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import open3d as o3d

client = carla.Client('localhost', 2000)
world = client.get_world()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

CAMERA_POS_Z = 2
CAMERA_POS_X = 2 

def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)


#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

# define speed contstants
PREFERRED_SPEED = 30 # what it says
PREFERRED_SPEED2 = 15 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 50


def collision_check(point, obstacles):
    collision_free = True
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<15:
            collision_free=False
            break

    return collision_free
def get_short_dist(point,obstacles):
    min_dist=float('inf')
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<min_dist:
            min_dist=collision_dist

    return min_dist

def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    c=math.cos(car.get_transform().rotation.yaw*math.pi/180)
    s=math.sin(car.get_transform().rotation.yaw*math.pi/180)
    for detection in point_cloud_data:
        #l.append([round(detection.point.x+car.get_location().x,1),round(detection.point.y+car.get_location().y,1)])
        #l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])
        l.append([round((detection.point.x *c +detection.point.y*-s)+car.get_location().x,1),round((detection.point.x*s+detection.point.y*c)+car.get_location().y,1)])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(1000))
    lidar_blueprint.set_attribute("range",str(50))
    lidar_blueprint.set_attribute("upper_fov",str(1))
    lidar_blueprint.set_attribute("lower_fov",str(-1))
#    lidar_blueprint.set_attribute("horizontal_fov",str(360))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 
def maintain_speed2(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED2:
        return 0
    elif s < PREFERRED_SPEED2 - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

wa=world.get_map().get_waypoint(carla.Location(x=-60,y=137,z=0.5))
start_point=wa.transform
start_point.location.z=0.5
midle_point=wa.transform
midle_point.location.z=0.5
midle_point.location.x=-50

vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle2 = world.try_spawn_actor(vehicle_bp[0], midle_point)


camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
camera_bp.set_attribute('image_size_x', '512') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '424')
camera_bp.set_attribute('fov', '70')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('depth Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('depth Camera',camera_data['image'])
POINTS = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

# Update geometry and camera in game loop
frame = 0
while True:
    
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    new_image=depth_to_array(image)
    POINTS.points=o3d.utility.Vector3dVector(depth_to_local_point_cloud(image,vehicle,max_depth=0.9))

    new_image2=new_image.copy()
    for x in range(len(new_image2)):
        for y in range(len(new_image2[x])):
            if new_image2[x][y]<0.2 and x<280:
                new_image2[x][y]=16

    min_val=float('inf')
    for x in range(len(new_image)):
        for y in range(len(new_image[x])):
            if new_image[x][y]<min_val and x<280:
                min_val=new_image[x][y]
    if frame == 2:
        vis.add_geometry(POINTS)
    vis.update_geometry(POINTS)
    
    vis.poll_events()
    vis.update_renderer()
    # # This can fix Open3D jittering issues:
    time.sleep(0.005)
    frame += 1

    #image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('depth Camera',new_image2)

cv2.destroyAllWindows()
camera.stop()
vis.destroy_window()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [11]:
o3d.__version__

'0.17.0'